Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

## Импорты

In [9]:
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
# import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML
%matplotlib inline

In [2]:
# в новой версии кераса можно использовать разные бекэнды, можно попробовать торч
# если заменить на tensorflow или jax то код также будет работать
# но нужно заранее установить нужный фреймворк

import os
os.environ["KERAS_BACKEND"] = "torch"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
# torch.set_default_device('cpu')

import keras
print(keras.__version__)

3.5.0


In [81]:
import re
from tqdm import tqdm

# !pip install pymorphy3
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

# !pip install gensim
import gensim

from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression as LogReg

## Данные wiki и базовый препроцессинг

In [3]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt

--2025-01-20 10:48:03--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/word_embeddings/wiki_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68582461 (65M) [text/plain]
Saving to: ‘wiki_data.txt’

wiki_data.txt       100%[===================>]  65.41M   324MB/s    in 0.2s    

2025-01-20 10:48:05 (324 MB/s) - ‘wiki_data.txt’ saved [68582461/68582461]



In [4]:
wiki = open('wiki_data.txt').read().split('\n')
len(wiki)

20003

In [27]:
class FasterMorphology:
    """ Класс для быстрого морфологического анализа текстов и их векторизации.
    Из курса по ООП.
    """

    def __init__(self) -> None: # Функция инициализации объекта после его создания.
        self.morpho = pymorphy3.MorphAnalyzer()
        self.__cash = {}
        self.__dictionary = {} # Добавим словарь для запоминания, на каком месте вектора находится какая начальная форма.

    def analyzeWords(self, words: list) -> list:
        """ Проводит морфологический анализ списка токенов words.
            Возвращает список начальных форм слов.
        """
        res: list

        res = []
        for w in words:
            if w in self.__cash: # Сперва ищем очередное слово в кеше.
                res.append(self.__cash[w])
            else: # Если его там нет, проводим морфологический анализ и кешируем.
                r = self.morpho.parse(w)[0].normal_form
                res.append(r)
                self.__cash[w] = r
                if r not in self.__dictionary: # Также для каждой начальной формы запоминаем ее позицию в векторе.
                    self.__dictionary[r] = len(self.__dictionary) + 1
        return res

faster_morph = FasterMorphology()

In [28]:
def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]

    # lemmatization
    tokens = faster_morph.analyzeWords(tokens)

    return tokens

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
1. добавьте лемматизацию в предобработку (любым способом)  
2. измените размер окна в большую или меньшую сторону
3. измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре).

Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

## Подготовка данных

In [5]:
cv = CountVectorizer(max_features=10000)
svd = TruncatedSVD(200)

X = cv.fit_transform(wiki)
X_svd = svd.fit_transform(X)

# Получившаяся матрица X нас не интересует. Нам нужно вытащить матрицу U, она лежит в svd.components_
# изначально U размерности (темы, слова) и для удобства ее нужно перевернуть - транспонировать
embeddings = svd.components_.T
embeddings.shape

(10000, 200)

In [8]:
# Теперь вытаскиваем соответствия слов индексам и наоборот
id2word = cv.get_feature_names_out()
word2id = {word:i for i,word in enumerate(id2word)}
word2id['птица']

7074

In [29]:
# Лучше сразу посчитать количество упоминаний, чтобы отсеять самые редкие.
vocab = Counter()

for text in tqdm(wiki):
    vocab.update(preprocess(text))

len(vocab)

100%|██████████| 20003/20003 [01:13<00:00, 273.33it/s]


262725

In [30]:
# Возьмем только те, что встретились больше 30 раз.
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

len(filtered_vocab)

12461

In [35]:
# Теперь нам нужно заменить в каждом тексте слова на числа (индексы в словаре).
# Создадим для этого специальный словарь с индексами.

word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

vocab_size = len(id2word)

In [34]:
# Заменяем слова на индексы.

sentences = []

for text in wiki:
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

print(sentences[:3])

[[1650, 8790, 6297, 6499, 9604, 9484, 3966, 8447, 1650, 8790, 10725, 9484, 8725, 12430, 9604, 63, 9484, 1193, 9647, 8494, 10405, 5577, 7442, 1228, 1193, 9647, 7905, 10405, 104, 3623, 10156, 8711, 9932, 8132, 8964, 9932, 2277, 7268, 8594, 7962, 3338, 4937, 12403, 9604, 4082, 1193, 7442, 4322, 4937, 12403, 9604, 2277, 10534, 1193, 1973, 5401, 4158, 11498, 3242, 1578, 416, 5507, 1973, 6297, 11179, 4286, 7204, 751, 1578, 1973, 6297, 3266, 8987, 9484, 8570, 1503, 6473, 5069, 11898, 5577, 7442, 63, 9029, 9484, 7029, 7442, 11721, 1193, 9647, 7905, 9484, 8181, 8447, 8486, 9484, 9604, 10726, 9548, 5491, 2312, 9484, 9604, 2277, 8740, 9604, 63, 7092, 10556, 11732, 10481, 11197, 7442, 7041, 9484, 9727, 9070, 9109, 3588, 1530, 2241, 8612, 1503, 9604, 1726, 8651, 11784, 9154, 8356, 3695, 7442, 827, 12215, 4087, 8967, 5069, 2926, 7442, 10627, 3219, 9484, 6489, 1503, 9484, 9604, 9154, 7997, 3695, 7442, 507, 12215, 5069, 10250, 7997, 1875, 12150, 3623, 1228, 1162, 11742, 1503, 9484, 9604, 4019, 4277, 3

## Обучение по SkipGram в парадигме negative sampling

In [36]:
# skip gram
                            # window = 4
def gen_batches_sg(sentences, window = 4, batch_size=1000):

    # параметр window задает его целиком
    # нам нужно поделить его пополам на левую и правую часть
    # когда делится неровно, то левая часть больше на 1
    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + \
                sent[i+1:i+right_context_length]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []


In [39]:
inputs_target = keras.layers.Input(shape=(1,))
inputs_context = keras.layers.Input(shape=(1,))

# размерность итоговых векторов (хотя 300 -- это канон, по заданию меняем)
out_dim = 250

embeddings_target = keras.layers.Embedding(
    input_dim=len(word2id), output_dim=out_dim)(inputs_target, )
embeddings_context = keras.layers.Embedding(
    input_dim=len(word2id), output_dim=out_dim)(inputs_context, )

target = keras.layers.Flatten()(embeddings_target)
context = keras.layers.Flatten()(embeddings_context)

dot = keras.layers.Dot(1)([target, context])
outputs = keras.layers.Activation(activation='sigmoid')(dot)

SkipGram_model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
SkipGram_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
SkipGram_model.fit(gen_batches_sg(sentences[:19000]),
          validation_data=gen_batches_sg(sentences[19000:]),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
          epochs=20) # Я прервал обучение после пятой эпохи для экономии,
                     # потому что качество менялось очень слабо

Epoch 1/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 123s 12ms/step - accuracy: 0.7879 - loss: 0.4628 - val_accuracy: 0.8230 - val_loss: 0.4116
Epoch 2/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 108s 11ms/step - accuracy: 0.8432 - loss: 0.3686 - val_accuracy: 0.8455 - val_loss: 0.3619
Epoch 3/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 117s 12ms/step - accuracy: 0.8457 - loss: 0.3612 - val_accuracy: 0.8530 - val_loss: 0.3523
Epoch 4/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 114s 11ms/step - accuracy: 0.8533 - loss: 0.3433 - val_accuracy: 0.8622 - val_loss: 0.3235
Epoch 5/20
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 121s 12ms/step - accuracy: 0.8550 - loss: 0.3395 - val_accuracy: 0.8416 - val_loss: 0.3725
Epoch 6/20
  811/10000 ━━━━━━━━━━━━━━━━━━━━ 1:37 11ms/step - accuracy: 0.8540 - loss: 0.3381

KeyboardInterrupt: 

## Обучение по CBOW в парадигме negative sampling

In [46]:
# cbow
                              # window = 4
def gen_batches_cbow(sentences, window = 4, batch_size=1000):

    # параметр window задает его целиком
    # нам нужно поделить его пополам на левую и правую часть
    # когда делится неровно, то левая часть больше на 1
    left_context_length = (window/2).__ceil__() # округлить в большую сторону
    right_context_length = window // 2 # округлить в меньшую сторону

    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-left_context_length):i] + \
                sent[i+1:i+right_context_length]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = keras.preprocessing.sequence.pad_sequences(
                        X_context, maxlen=window)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [47]:
inputs_target = keras.layers.Input(shape=(1,))
inputs_context = keras.layers.Input(shape=(4,))

# размерность итоговых векторов (хотя 300 -- это канон, по заданию меняем)
out_dim = 250

embeddings_target = keras.layers.Embedding(
    input_dim=len(word2id), output_dim=out_dim)(inputs_target, )
embeddings_context = keras.layers.Embedding(
    input_dim=len(word2id), output_dim=out_dim)(inputs_context, )

target = keras.layers.Flatten()(embeddings_target)
context = keras.layers.Lambda(lambda x: x.sum(axis=1))(embeddings_context)
dot = keras.layers.Dot(1)([target, context])

# полученную близость нужно преобразовать в вероятность
# когда она одна используется не софтмакс и сигмоида
outputs = keras.layers.Activation(activation='sigmoid')(dot)

CBOW_model = keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)

optimizer = keras.optimizers.Adam(learning_rate=0.001)
CBOW_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
CBOW_model.build([(None, 1), (None, 4)]) # не совсем понимаю, что это делает

CBOW_model.fit(gen_batches_cbow(sentences[:19000]),
          validation_data=gen_batches_cbow(sentences[19000:]),
          batch_size=1000,
          steps_per_epoch=5000,
          validation_steps=30,
          epochs=5) # взял по аналогии с предыдущим опытом и с семинаром; в нём
                    # с другими окном и длиной качество чуть лучше (на < 0.02)

Epoch 1/5
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 75s 15ms/step - accuracy: 0.8110 - loss: 0.4246 - val_accuracy: 0.8695 - val_loss: 0.3187
Epoch 2/5
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 80s 16ms/step - accuracy: 0.8745 - loss: 0.3041 - val_accuracy: 0.8906 - val_loss: 0.2694
Epoch 3/5
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 73s 15ms/step - accuracy: 0.8986 - loss: 0.2467 - val_accuracy: 0.8953 - val_loss: 0.2583
Epoch 4/5
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 79s 16ms/step - accuracy: 0.9084 - loss: 0.2242 - val_accuracy: 0.9039 - val_loss: 0.2406
Epoch 5/5
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 82s 16ms/step - accuracy: 0.9183 - loss: 0.2025 - val_accuracy: 0.8919 - val_loss: 0.2714


## Оценка результатов

In [49]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [50]:
SkipGram_embs = SkipGram_model.layers[2].get_weights()[0]
CBOW_embs = CBOW_model.layers[2].get_weights()[0]

In [53]:
print(*most_similar('радость', SkipGram_embs))
# видно, что попало стоп-слово "почему", в остальном нормально

радость ужасный гомер порой грех удивление счастие ненависть почему кронос


In [55]:
print(*most_similar('сосна', SkipGram_embs))

сосна дуб хвойный липа берёза лиственный клён ель пихта берёзовый


In [56]:
print(*most_similar('радость', CBOW_embs))
# результаты хуже, чем у СкипГрама, какой-то прекос в "мистику"

радость отшельник удовольствие ева уверенность незнакомец оракул аннабет плодородие твой


In [57]:
print(*most_similar('сосна', CBOW_embs))
# в этом случае неплохо

сосна хвойный клён ель бизон берёза тополь поляна bison лиственный


# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [65]:
texts = [preprocess(text) for text in wiki]
print(*texts[0][:10])

новостройка нижегородский область новостро́йка — сельский посёлок в дивеевский район


In [68]:
%%time
w2v = gensim.models.Word2Vec(texts,
                             vector_size=250,
                             min_count=50,
                             max_vocab_size=7500,
                             window=5,
                             epochs=5,
                             sg=1, # hence cbow_mean is unapplicable
                             hs=0,
                             negative=10,
                             sample=1e-4,
                             ns_exponent=0.8
                             )
# Хотя дефолтная настройка работает достаточно быстро и хорошо

CPU times: user 3min 17s, sys: 525 ms, total: 3min 18s
Wall time: 2min 12s


In [72]:
w2v.wv.most_similar('школа')

[('учиться', 0.6460990905761719),
 ('колледж', 0.6183080077171326),
 ('училище', 0.6115320920944214),
 ('учитель', 0.5766366720199585),
 ('детский', 0.5424436330795288),
 ('окончить', 0.5225055813789368),
 ('обучение', 0.5174224376678467),
 ('факультет', 0.5128446221351624),
 ('учебный', 0.5110342502593994),
 ('университет', 0.5050925016403198)]

In [73]:
%%time
ft = gensim.models.FastText(texts, min_n=4, max_n=9)

CPU times: user 5min 24s, sys: 1.13 s, total: 5min 25s
Wall time: 3min 19s


In [74]:
ft.wv.most_similar('школа')

[('школа»', 0.9957459568977356),
 ('«школа', 0.9952937960624695),
 ('школы»', 0.9852859973907471),
 ('эркола', 0.9781908392906189),
 ('школе»', 0.9713997840881348),
 ('«школа»', 0.9686146378517151),
 ('анкола', 0.9636104106903076),
 ('кока-кола', 0.9457550048828125),
 ('лола', 0.9444689154624939),
 ('бизнес-школа', 0.9393897652626038)]

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.

В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)

Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор.

В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)

F1 мера должна быть выше 20%.

In [77]:
data = pd.read_csv('labeled.csv')

In [78]:
data['norm_text'] = data.comment.apply(preprocess) # including lemmatization
data.head()

,comment,toxic,norm_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюд-то, за, что, дебил, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохол, это, отдушина, затюканый, россиянин, м..."
2,Собаке - собачья смерть\n,1.0,"[собака, , собачий, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страница, обновить, дебил, это, тоже, не, оск..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[ты, не, убедить, 6-страничный, пдф, в, тот, ч..."


In [107]:
y = data.toxic.values

X = []
for text in data['norm_text']:
    text_emb = np.zeros(w2v.vector_size)
    for token in text:
        if token in w2v.wv:
            text_emb += w2v.wv[token]
    X.append(text_emb / len(text)) # хотя в общем случае количество учтённых векторов меньше

In [109]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [110]:
%%time
clf = LogReg()
clf.fit(X_train, y_train)

CPU times: user 315 ms, sys: 58 ms, total: 373 ms
Wall time: 215 ms


LogisticRegression()

In [111]:
preds = clf.predict(X_valid)
print(classification_report(y_valid, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.79      0.91      0.84       472
         1.0       0.75      0.53      0.62       249

    accuracy                           0.78       721
   macro avg       0.77      0.72      0.73       721
weighted avg       0.77      0.78      0.77       721



In [113]:
preds_proba = clf.predict_proba(X_valid)
c = Counter({text: prob[1] for text, prob in zip(data.comment, preds_proba)})
c.most_common(5)
# классификатор токсичности работает; тексты некорректные и ненавистнические

[('Это у тебя подгорает в одном месте, видимо перцем намазали....\n',
  0.9913489651621333),
 ('Вы вот смеетесь, а что будет, когда у нас эти смешные хохлы закончатся?\n',
  0.984260629149829),
 ('Я, дурачок, купил. Игра гавно, однообразно как и все последние высеры биотех, упоминание в посте Dragon age - origins, кощунственно.\n',
  0.9787731441094208),
 ('Правильно разбомбили сербских уродов. Жаль что не истребили их всех\n',
  0.9758253271213012),
 ('А ещё эта пчела - расист )\n', 0.9663071257357857)]

In [118]:
c.most_common(721)[-5:-1]
# впрочем, как и видно из отчёта выше, есть ошибки классификации

[('Нет, пожалуй, отвечу развернутее. Представь, что тебя ограбили. Ты идешь в частную полицию, выкладываешь бабки на стол, говоришь о случившемся. Полицаи по мере своих сил расследуют инцидент. Они дают тебе список потенциальных преступников, вероятнее всего, из одного человека. Ты приходишь к человеку, говоришь, так и так, чувак, я тебя подозреваю, хочу судиться. Если он тоже хочет судиться, то он идет с тобой в выбранный вами обоими суд. Если не хочет, то ты обращаешься в суд со своей доказательной базой, он приходит к человеку и обращается с подобной фразой: вы, гражданин такой-то такой-то, обвиняетесь по тому-то тому-то. Пройдите с нами в зал суда для разбирательств, или мы, согласно статье такой-то такой-то действующего законодательства имеем право вас наказать. Если человек соглашается, то вы судитесь. Если он виновен, то ему назначается наказание и он оплачивает издержки суда, если нет, то ты оплачиваешь издержки суда и идешь искать преступника дальше силами частной полиции. Зву

# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

In [75]:
pass